In [ ]:
# dependencies import
from common_dependencies import *
import model_versions as mv
import time
logger = logging.getLogger(f'main.model_train')
# детерминация случайных величин, отвечающих за выбор первоначальных весов и биасов

In [ ]:
# train params that can be easily changed
tf.compat.v1.set_random_seed(290)
tf.random.set_seed(290)
np.random.seed(290)

CROP_SIZE = 16
#CROP_STEP = 16
MAX_VAL = 1000
XSHIFT = 200
MODEL_VER = '14'
#OLD_MODEL_NUM = '03'
NEW_MODEL_NUM = '05'
PATH_TO_SAVE_CHECKPOINTS = f"networks/CNN/checkpoints/model_id=v{MODEL_VER}n{NEW_MODEL_NUM}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)"
PATH_TO_SAVE_LOGS = f"networks/CNN/logs/model_id=v{MODEL_VER}n{NEW_MODEL_NUM}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)"

if not os.path.exists(PATH_TO_SAVE_CHECKPOINTS):
        os.makedirs(PATH_TO_SAVE_CHECKPOINTS)

if not os.path.exists(PATH_TO_SAVE_LOGS):
        os.makedirs(PATH_TO_SAVE_LOGS)
    
dataset_desc = {'train': (
                            DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,140,MAX_VAL),SlidingCrop(CROP_SIZE,4),XSHIFT),
                            DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(140,0,120,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),
                            DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(260,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,4),XSHIFT),
                         ),

                'val': (
                            DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(195,0,85,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),
                            DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(280,0,MAX_VAL,MAX_VAL),SlidingCrop(CROP_SIZE,4),XSHIFT),
                
                        ),
                'test': (
                            DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,110,MAX_VAL),SlidingCrop(CROP_SIZE,4),XSHIFT),
                            DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(110,0,85,MAX_VAL),SlidingCrop(CROP_SIZE,1),XSHIFT),
                
                        )
               }

In [ ]:
# display info about the dataset used in model training

dataset = {'train':dict(zip(['x','y','bin'], chain_dataset_gens(dataset_desc['train']))),
           'val':dict(zip(['x','y','bin'], chain_dataset_gens(dataset_desc['val']))),
           'test':dict(zip(['x','y','bin'], chain_dataset_gens(dataset_desc['test'])))}

# shape displaying
print('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    print('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        item_shape = next(data_part).shape if type(next(data_part)) == np.ndarray else '(1)'
        print(f'{data_part_name} items total: ({sum(1 for x in data_part)+1}). Item shape: {item_shape}')

#выборка данных
# show parts took for learning
all_rects = {'run_1': {'train':None,'val':None,'test':None}, 
             'run_2': {'train':None,'val':None,'test':None}}
rects_colors = {'train':'red', 'val':'green', 'test':'yellow'}

for run_name in all_rects.keys():
    x_df, y_df = dataset.get_x_and_y_data_dfs(dw.DataPart(path_to_run_folder=PATH_TO_DATA[run_name]))
    x_df = None
    y_df = dw.roll_df(y_df, XSHIFT, 1)
    for dataset_part_name in all_rects[run_name].keys():
        # get all DatasetPartDescription for train, val or test
        dataset_part_desc = dataset_desc[dataset_part_name]
        # get all DatasetPartDescription for current run_name (run_1 or run_2)
        dataset_part_desc = [dataset_part for dataset_part in dataset_part_desc if re.findall(r'run_\d', dataset_part.data_path_tuple[0])[0] == run_name]
        # put rects list to all_rects[run_name][dataset_part_name]
        all_rects[run_name][dataset_part_name] = [Rectangle((dataset_part.file_data_crop.left, dataset_part.file_data_crop.top), 
                           dataset_part.file_data_crop.width, dataset_part.file_data_crop.height, 
                           facecolor=rects_colors[dataset_part_name], alpha=0.4, edgecolor='white')
                                                  for dataset_part in dataset_part_desc]
        
    res_rects = list(itertools.chain(*[run_rects for run_rects_name, run_rects in all_rects[run_name].items()]))
    if res_rects:
        dw.draw_defects_map_with_rectangles_owerlap(y_df, res_rects, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')
    else:
        dw.draw_defects_map(y_df, title = f'The parts took for learning from {run_name} (red - train, green - validate, yellow - test)')

In [ ]:
#### всякие константы для последующей работы

#///////////////////////////////// для компиляции 

CMP_learning_rate = 0.0005 #0.0000002 # шаг сходимости back propogation
#CMP_solver = keras.optimizers.Adam(CMP_learning_rate) # оптимизатор
CMP_solver = keras.optimizers.SGD(CMP_learning_rate) # оптимизатор
CMP_loss_func = keras.losses.BinaryCrossentropy() #BinaryCrossentropy() 
CMP_metrics = [keras.metrics.BinaryCrossentropy(name='loss'),
               keras.metrics.BinaryAccuracy(name='BinaryAccuracy'),
               keras.metrics.MeanSquaredError(name='MSE'),
               #keras.metrics.TruePositives(name='TruePositives'),
               #keras.metrics.FalsePositives(name='FalsePositives'),
               #keras.metrics.TrueNegatives(name='TrueNegatives'),
               #keras.metrics.FalseNegatives(name='FalseNegatives'),     
               keras.metrics.Precision(name='Precision'),
               keras.metrics.Recall(name='Recall'),
               keras.metrics.AUC(name='PR_AUC', curve='PR')
              ]
#///////////////////////////////// для колбэков

'''# для Early_stopping
ES_patience = 6 # кол-во эпох без улучшений
ES_min_delta = 0.00001 # минимальное улучшение параметра за cur_patience
ES_monitor_parametr =  'loss' # отслеживаемый параметр 
ES_save_best_weights = False # сохранять ли веса нейронки с лучшими результатами
Es_mode = 'max'

   # для ReduceLROnPlateau
RLPOP_monitor_parametr = 'loss'  # отслеживаемый параметр 
RLPOP_factor = 0.3 # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
RLPOP_patience = 1 # кол-во эпох без улучшений
RLPOP_verbose = 1 # выводить ли прогресс изменения шага сходимости в его процессее
RLPOP_mode = 'auto' # выбирает, уменьшать шаг сходимости при росте величины или при её уменьшении
RLPOP_min_delta = 0.0001 # порог изменения отслеживаемого значения
RLPOP_cooldown = 3 # количество эпох до возобновления работы после изменения шага сходимости
RLPOP_min_lr = 0 # минимальное значение шага сходимости

    # для ModelCheckpoint
MC_path = f"networks/CNN/checkpoints/model_id=v{MODEL_VER}n{NEW_MODEL_NUM}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)" + "/check-{epoch:04d}.keras"

    # для CallbackList
CBL_add_history = True # вызывать ли колбэк History (если он не был довавлен вручную)
CBL_add_progbar = False # вызывать ли колбэк ProgbarLogger (если он не был довавлен вручную)

    # для TensorBoard
TB_log_dir = f"networks/CNN/logs/model_id=v{MODEL_VER}n{NEW_MODEL_NUM}/" # путь для сохранения логов процесса обучения модели
TB_histogram_freq = 1 # частота сохранения логов'''

#///////////////////////////////// для тренировки

FIT_minibatch_size = 1 # для какого кол-ва наблюдений считать back propagation за раз
FIT_buffer_size = FIT_minibatch_size * 2000 # сколько взять данных из генератора чтобы это влезло в оперативную память пк и видеокарты
#FIT_shuffle = True # перемешивать ли данные
FIT_verbose = True # выводить ли прогресс обучения в его процессее
FIT_epochs = 4 # количество эпох обучения

In [ ]:
#model = mv.get_model_v10(CROP_SIZE)
'''DROP = 0.1

def bnorm(layer, drop_percent):
    #return Dropout(drop_percent)(BatchNormalization()(layer))
    return Dropout(drop_percent)(layer)
    #return BatchNormalization()(layer)'''

augment_data = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  #layers.RandomRotation(1, fill_mode='reflect'),
  layers.RandomTranslation(0.2,0.2, fill_mode="reflect"),
  #layers.RandomZoom(0.3,0.3, fill_mode="reflect")
])

input_data = Input((16,16,64), name = 'input_data')

aug_data = augment_data(input_data)

# 12
dconv_1_1 = Conv2D(128, (3,3), dilation_rate=(2, 2), activation='relu', name='dconv_1_1')(aug_data)

# 6
dconv_1_2 = Conv2D(128, (3,3), dilation_rate=(5, 5), activation='relu', name='dconv_1_2')(aug_data)
up_1_2 = UpSampling2D(2, interpolation='bilinear', name='up_1_2') (dconv_1_2)

# 4
dconv_1_3 = Conv2D(128, (3,3), dilation_rate=(6, 6), activation='relu', name='dconv_1_3')(aug_data)
up_1_3 = UpSampling2D(3, interpolation='bilinear', name='up_1_3') (dconv_1_3)

# 2
dconv_1_4 = Conv2D(128, (3,3), dilation_rate=(7, 7), activation='relu', name='dconv_1_4')(aug_data)
up_1_4 = UpSampling2D(6, interpolation='bilinear', name='up_1_4') (dconv_1_4)

# 12
conc_1_1 = concatenate([dconv_1_1, up_1_2, up_1_3, up_1_4],axis=3, name='conc_1_1')

conv_2_1 = Conv2D(512, (3,3), dilation_rate=(2, 2), activation='relu', padding='same', name='conv_2_1')(conc_1_1)
conv_2_2 = Conv2D(512, (3,3), dilation_rate=(2, 2), activation='relu', padding='same', name='conv_2_2')(conv_2_1)
conv_2_3 = Conv2D(512, (3,3), dilation_rate=(2, 2), activation='relu', padding='same', name='conv_2_3')(conv_2_2)
pool_2_1 = MaxPooling2D((2,2), strides=2, name='pool_2_1')(conv_2_3)

conv_2_4 = Conv2D(1024, (2,2), activation='relu', name='conv_2_4')(pool_2_1) # core (2,2)
conv_2_5 = Conv2D(1024, (2,2), activation='relu', name='conv_2_5')(conv_2_4)
conv_2_6 = Conv2D(1024, (2,2), activation='relu', name='conv_2_6')(conv_2_5)
conv_2_7 = Conv2D(1024, (2,2), activation='relu', name='conv_2_7')(conv_2_6)
pool_2_2 = MaxPooling2D((2,2), strides=2, name='pool_2_2')(conv_2_7)

d_4_1 = Dense(1024, activation='relu', name='d_4_1')(Flatten(name='flat_3_1')(pool_2_2))
d_4_2 = Dense(512, activation='relu', name='d_4_2')(d_4_1)
d_4_3 = Dense(256, activation='relu', name='d_4_3')(d_4_2)
d_4_4 = Dense(128, activation='relu', name='d_4_4')(d_4_3)
d_4_5 = Dense(64, activation='relu', name='d_4_5')(d_4_4)
d_4_6 = Dense(16, activation='relu', name='d_4_6')(d_4_5)
d_4_7 = Dense(4, activation='relu', name='d_4_7')(d_4_6)

output_def_bool = Dense(1, activation='sigmoid', name='output_def_bool')(d_4_7)
    
model = keras.Model([aug_data], [output_def_bool], name=f'id_v{MODEL_VER}n{NEW_MODEL_NUM}')

#model = keras.models.load_model(f"networks/CNN/id=v{MODEL_VER}n{OLD_MODEL_NUM}_in({CROP_SIZE}x{CROP_SIZE}x64)_out(1)_train=0dot0_test=0dot0.keras")

#model.compile(optimizer=CMP_solver, loss=CMP_loss_func, metrics=CMP_metrics)

print(model.summary())

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)

In [ ]:
'''# Создание и настройка колбэков
callback_list = [] # массив колбэков до подачи в колбек "callbacklist"

callback_list.append(keras.callbacks.EarlyStopping(
            monitor = ES_monitor_parametr, 
            min_delta = ES_min_delta, 
            patience = ES_patience,
            restore_best_weights = ES_save_best_weights
            ))

callback_list.append(keras.callbacks.ModelCheckpoint(
            MC_path, 
            verbose=1, 
            save_weights_only=False,
            save_freq='epoch'))

callback_list.append(keras.callbacks.ReduceLROnPlateau(
            monitor = RLPOP_monitor_parametr, 
            factor = RLPOP_factor, 
            patience = RLPOP_patience, 
            verbose = RLPOP_verbose,
            mode = RLPOP_mode, 
            min_delta = RLPOP_min_delta, 
            cooldown = RLPOP_cooldown, 
            min_lr = RLPOP_min_lr
            ))

callback_list.append(keras.callbacks.TensorBoard(
            log_dir=TB_log_dir, 
            histogram_freq=TB_histogram_freq))

FIT_callback_list = keras.callbacks.CallbackList(
            callbacks = callback_list, 
            add_history = CBL_add_history, 
            add_progbar = CBL_add_progbar, 
            model = model
            )''';

In [ ]:
def get_batched_array(arr: np.ndarray, bach_size: int):
    for i in range(0, arr.shape[0], bach_size):
        yield arr[i:i+bach_size]

@tf.function
def train_batch_step(train_batch_x, train_batch_y, train_batch_bin, model, loss, solver, metrics):
    # bach propagation on minibatch
    with tf.GradientTape() as tape:
        logits = model(train_batch_x, training=True)  # Logits for this minibatch
        loss_value = loss(train_batch_bin, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    solver.apply_gradients(zip(grads, model.trainable_weights))
    
    # metrics update
    for metric in CMP_metrics:
        metric.update_state(train_batch_bin, logits)

@tf.function
def val_batch_step(val_batch_x, val_batch_y, val_batch_bin, model, loss, solver, metrics):
    # bach propagation on minibatch
    with tf.GradientTape() as tape:
        logits = model(val_batch_x, training=True)  # Logits for this minibatch
        loss_value = loss(val_batch_bin, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    solver.apply_gradients(zip(grads, model.trainable_weights))
    
    # metrics update
    for metric in CMP_metrics:
        metric.update_state(val_batch_bin, logits)

In [ ]:
logs = {metric.get_config()["name"]:[] for metric in CMP_metrics} | {f'val_{metric.get_config()["name"]}':[] for metric in CMP_metrics}
for epoch in range(FIT_epochs):
    start_time = time.time()
    # get dataset generator for each epoch
    dataset = {'train':dict(zip(['x','y','bin'], [dw.get_batch_generator(part,FIT_buffer_size) for part in chain_dataset_gens(dataset_desc['train'])] )),
               'val':dict(zip(['x','y','bin'], [dw.get_batch_generator(part,FIT_buffer_size) for part in chain_dataset_gens(dataset_desc['val'])] ))}
    # |||||||||||||||||||||||||||||||||||||||||||| train ||||||||||||||||||||||||||||||||||||||||||||
    print(f'Epoch: {epoch+1}/{FIT_epochs}')
    for train_buffer_x, train_buffer_y, train_buffer_bin in zip(dataset['train']['x'],dataset['train']['y'],dataset['train']['bin']):
        for train_batch_x, train_batch_y, train_batch_bin in zip(get_batched_array(train_buffer_x, FIT_minibatch_size), 
                                                                 get_batched_array(train_buffer_y, FIT_minibatch_size), 
                                                                 get_batched_array(train_buffer_bin, FIT_minibatch_size)):
            #bach propagation on minibatch
            train_batch_step(train_batch_x, train_batch_y, train_batch_bin, model, CMP_loss_func, CMP_solver, CMP_metrics)
            
    print('TRAIN',end='\t')        
    for metric in CMP_metrics:
        logs[metric.get_config()["name"]].append(metric.result().numpy())
        print(f'{metric.get_config()["name"]}:{metric.result():.6f}', end=' ')
        metric.reset_state()
    print()
    
    # |||||||||||||||||||||||||||||||||||||||||||| val ||||||||||||||||||||||||||||||||||||||||||||
    # Run a validation loop at the end of each epoch.
    for val_buffer_x, val_buffer_y, val_buffer_bin in zip(dataset['val']['x'],dataset['val']['y'],dataset['val']['bin']):
        for val_batch_x, val_batch_y, val_batch_bin in zip(get_batched_array(val_buffer_x, FIT_minibatch_size), 
                                                           get_batched_array(val_buffer_y, FIT_minibatch_size), 
                                                           get_batched_array(val_buffer_bin, FIT_minibatch_size)):
        
            val_batch_step(val_batch_x, val_batch_y, val_batch_bin, model, CMP_loss_func, CMP_solver, CMP_metrics)
            
    print('VAL',end='\t')     
    for metric in CMP_metrics:
        logs[f'val_{metric.get_config()["name"]}'].append(metric.result().numpy())
        print(f'{metric.get_config()["name"]}:{metric.result():.6f}', end=' ')
        metric.reset_state()

    model.save(f"{PATH_TO_SAVE_CHECKPOINTS}/check-{epoch:04}.keras")
    print()
    print(f"Time taken: {(time.time() - start_time):.2f}")

In [ ]:
# сохранинеи процесса обучения в файл
learn_df = pd.DataFrame.from_dict(logs)
learn_df.to_excel(f'{PATH_TO_SAVE_LOGS}/learn_df.xlsx')
display(learn_df)

In [ ]:
FONT_SIZE = 15
for key1,key2 in zip(['BinaryAccuracy','Precision','MSE'],
                     ['loss','Recall','PR_AUC']):
    fig, axes = plt.subplots(1,2)

    fig.set_figwidth(22)
    fig.set_figheight(8)
    
    axes[0].plot(logs[key1], 
             label='Train dataset',  linewidth=1.5, color='blue')
    axes[0].plot(logs[f'val_{key1}'], linestyle = '--', 
             label='Validation dataset',  linewidth=3, color='red')
    axes[0].set_xlabel('Epoch number', fontsize=FONT_SIZE)
    axes[0].set_ylabel(f'{key1} value', fontsize=FONT_SIZE)
    axes[0].set_title(f"Learning process {key1} plot", fontsize=FONT_SIZE, pad=15)
    axes[0].tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    axes[0].minorticks_on()
    axes[0].grid(which='major', linewidth=2)
    axes[0].grid(which='minor', color = 'gray', linestyle = ':')

    axes[1].plot(logs[key2], 
             label='Train dataset',  linewidth=1.5, color='blue')
    axes[1].plot(logs[f'val_{key2}'], linestyle = '--', 
             label='Validation dataset',  linewidth=3, color='red')
    axes[1].set_xlabel('Epoch number', fontsize=FONT_SIZE)
    axes[1].set_ylabel(f'{key2} value', fontsize=FONT_SIZE)
    axes[1].set_title(f"Learning process {key2} plot", fontsize=FONT_SIZE, pad=15)
    axes[1].tick_params(axis='both', which='both', labelsize = FONT_SIZE)
    axes[1].minorticks_on()
    axes[1].grid(which='major', linewidth=2)
    axes[1].grid(which='minor', color = 'gray', linestyle = ':')

    plt.show()

In [ ]:
# quick check is tf.dataDataset is working in current tensorflow version
'''dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])
for element in dataset:
  print(element)''';

In [ ]:
# RIGHT WAY to use generators in tf.data.Dataset instead of manual method on tensorflow level
'''def train_dataset(epochs, batch, crop_size):
    (train_x_time, 
     train_x_amp, 
     train_y_binary,
     y_data_depth) = get_dataset_gen(PATH_TO_DATA['run_1'], 200, DataCrop(None,None,200,None), crop_size, 16)
    
    (train_x_time2, 
    train_x_amp2, 
    train_y_binary2,
    y_data_depth2) = get_dataset_gen(PATH_TO_DATA['run_2'], 200, DataCrop(None,None,None,200), crop_size, 2)
    
    train_x_time = itertools.chain(train_x_time1,train_x_time2)
    train_x_amp = itertools.chain(train_x_amp1,train_x_amp2)
    train_y_binary = itertools.chain(train_y_binary1,train_y_binary2)
    y_data_depth = itertools.chain(y_data_depth1,y_data_depth2)
    
    def my_generator(x_time_gen, x_amp_gen, y_binary_gen):
        for time,amp,binary in zip(x_time_gen, x_amp_gen, y_binary_gen):
            yield (time, amp), binary
            
    dataset = tf.data.Dataset.from_generator(lambda: my_generator(train_x_time, train_x_amp, train_y_binary),
                                             output_signature=(
                                                 (tf.TensorSpec(shape=(crop_size,crop_size,32), dtype=tf.float64),
                                                  tf.TensorSpec(shape=(crop_size,crop_size,32), dtype=tf.float64)),
                                                  tf.TensorSpec(shape=(), dtype=tf.bool)))

    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch)
    
    return dataset'''

'''def val_dataset(epochs, batch, crop_size):
    (val_x_time, 
     val_x_amp, 
     val_y_binary,
     y_data_depth) = get_dataset_gen(PATH_TO_DATA['run_1'], 200, DataCrop(None,None,None,200), crop_size, 2)
    
    def my_generator(x_time,x_amp,y_binary):
        for time,amp,binary in zip(val_x_time, val_x_amp, val_y_binary):
            yield (time, amp), binary
            
    dataset = tf.data.Dataset.from_generator(lambda: my_generator(val_x_time, val_x_amp, val_y_binary),
                                             output_signature=(
                                                 (tf.TensorSpec(shape=(crop_size,crop_size,32), dtype=tf.float64),
                                                  tf.TensorSpec(shape=(crop_size,crop_size,32), dtype=tf.float64)),
                                                  tf.TensorSpec(shape=(), dtype=tf.bool)))

    dataset = dataset.repeat(epochs)
    dataset = dataset.batch(batch)
    
    return dataset''';

#### 